- consider only xy in validation
- bo back to version 6 lstm
- reset indoor-lgb-floor prediction (version 5) *
- change activations in lstm layers *

In [1]:
# credit: https://www.kaggle.com/kokitanisaka/lstm-by-keras-with-unified-wi-fi-feats
!pip install tensorflow-determinism

  Created wheel for tensorflow-determinism: filename=tensorflow_determinism-0.1.0-py3-none-any.whl size=5255 sha256=67ac643a14f74e9463b33320bc75b56f6a51bef823157dbcf45aecbd6d13dbb8
  Stored in directory: /root/.cache/pip/wheels/ce/06/d4/df757adc4c81f705a74a0317c27cf19919ccd25ae2a6ecd2c5
Successfully built tensorflow-determinism


In [2]:
!git clone --depth 1 https://github.com/location-competition/indoor-location-competition-20 indoor_location_competition_20
!rm -rf indoor_location_competition_20/data

Cloning into 'indoor_location_competition_20'...
remote: Enumerating objects: 1169, done.
remote: Counting objects: 100% (1169/1169), done.
remote: Compressing objects: 100% (1131/1131), done.
remote: Total 1169 (delta 38), reused 1167 (delta 38), pack-reused 0
Receiving objects: 100% (1169/1169), 411.37 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Checking out files: 100% (1145/1145), done.


In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob
import pickle

import random
import os

from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *

import multiprocessing
import scipy.interpolate
import scipy.sparse
from tqdm import tqdm

from indoor_location_competition_20.io_f import read_data_file
import indoor_location_competition_20.compute_f as compute_f

from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

In [4]:
# options
N_SPLITS = 5
SEED = 2021
NUM_FEATS = 30 # number of features that we use. there are 100 feats but we don't need to use all of them
base_path = '/kaggle'

In [5]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def xy_comp_metric(xhat, yhat, x, y):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2))
    return intermediate.sum()/xhat.shape[0]

In [6]:
feature_dir = f"{base_path}/input/indooruniteddataset"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
subm = pd.read_csv(f'{base_path}/input/indoor-location-navigation/sample_submission.csv', index_col=0)

all_sites = sorted(set(pd.DataFrame(subm.index)["site_path_timestamp"].apply(lambda x: x.split("_")[0])))

In [7]:
floor_train_pred = pd.read_csv(f'{base_path}/input/indoorlgbfloor/floor_prediction_in_train.csv')
floor_train_only_pred = pd.read_csv(f'{base_path}/input/indoorlgbfloor/floor_prediction_in_train_only.csv')
floor_test_pred = pd.read_csv(f'{base_path}/input/indoorlgbfloor/submission.csv')

In [8]:
# training target features
BSSID_FEATS = [f'bssid_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS  = [f'rssi_{i}' for i in range(NUM_FEATS)]

In [9]:
data = []
for i in tqdm(train_files):
    tmp = pd.read_csv(i)
    tmp["site_id"] = i.split("/")[4].split("_")[0]
    data.append(tmp)
data = pd.concat(data).reset_index(drop=True)

test_data = []
for i in tqdm(test_files):
    tmp = pd.read_csv(i)
    tmp["site_id"] = i.split("/")[4].split("_")[0]
    test_data.append(tmp)
test_data = pd.concat(test_data).reset_index(drop=True)

100%|██████████| 24/24 [00:00<00:00, 24.57it/s]


In [10]:
#train_only_index = data[~data.site_id.isin(all_sites)].index
#normal_index = data[data.site_id.isin(all_sites)].index
#assert len(set(train_only_index) & set(normal_index)) == 0

In [11]:
wifi_feat_nums = 60
wifi_bssids = []
for i in range(wifi_feat_nums):
    wifi_bssids.extend(data.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids_test = []
for i in range(wifi_feat_nums):
    wifi_bssids_test.extend(test_data.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids.extend(wifi_bssids_test)
wifi_bssids_size = len(wifi_bssids)

BSSID TYPES: 54067
BSSID TYPES: 25654


In [12]:
# preprocess

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(data['site_id'])

ss = StandardScaler()
ss.fit(data.loc[:,RSSI_FEATS+['floor']])

StandardScaler()

In [13]:
data.loc[:,RSSI_FEATS+['floor']] = ss.transform(data.loc[:,RSSI_FEATS+['floor']])
for i in BSSID_FEATS:
    data.loc[:,i] = le.transform(data.loc[:,i])
    
data.loc[:, 'site_id'] = le_site.transform(data.loc[:, 'site_id'])

data.loc[:,RSSI_FEATS+['floor']] = ss.transform(data.loc[:,RSSI_FEATS+['floor']])

In [14]:
lgb_pred = pd.read_csv('../input/indoorlgbfloor/submission.csv')
test_data['floor'] = lgb_pred['floor'].values

In [15]:
test_data.loc[:,RSSI_FEATS+['floor']] = ss.transform(test_data.loc[:,RSSI_FEATS+['floor']])
for i in BSSID_FEATS:
    test_data.loc[:,i] = le.transform(test_data.loc[:,i])
    
test_data.loc[:, 'site_id'] = le_site.transform(test_data.loc[:, 'site_id'])

test_data.loc[:,RSSI_FEATS+['floor']] = ss.transform(test_data.loc[:,RSSI_FEATS+['floor']])

In [16]:
site_count = len(data['site_id'].unique())
data.reset_index(drop=True, inplace=True)

In [17]:
set_seed(SEED)

In [18]:
#train_only_data = data[data.index.isin(train_only_index)].copy().reset_index(drop=True)
#data = data[data.index.isin(normal_index)].copy().reset_index(drop=True)
#data.shape, train_only_data.shape

# model

In [19]:
num_heads = 8
bssid_embed_dim = 64
site_embed_dim = 2
ff_dim = 128 
DROPOUT = 0.2
maxlen = NUM_FEATS
batch_size=256
epochs= 40

In [20]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=DROPOUT):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [21]:
def create_model(input_data):

    # bssid feats
    input_dim = input_data[0].shape[1]

    input_embd_layer = L.Input(shape=(input_dim,))
    x1 = L.Embedding(wifi_bssids_size, bssid_embed_dim)(input_embd_layer)
    x1 = L.Flatten()(x1)

    # rssi feats
    input_dim = input_data[1].shape[1]

    input_layer = L.Input(input_dim, )
    x2 = L.BatchNormalization()(input_layer)
    x2 = L.Dense(NUM_FEATS * 64, activation='relu')(x2)

    # site
    input_site_layer = L.Input(shape=(1,))
    x3 = L.Embedding(site_count, site_embed_dim)(input_site_layer)
    x3 = L.Flatten()(x3)

    
    # main stream
    x = L.Concatenate(axis=1)([x1, x3, x2])

    x = L.BatchNormalization()(x)
    x = L.Dropout(0.3)(x)
    x = L.Dense(256, activation='relu')(x)

    x = L.Reshape((1, -1))(x)
    x = L.BatchNormalization()(x)
    x = L.LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, activation='elu')(x)
    x = L.LSTM(16, dropout=0.1, return_sequences=False, activation='elu')(x)
    
    output_layer_1 = L.Dense(2, name='xy')(x)
    output_layer_2 = L.Dense(1, activation='softmax', name='floor')(x)

    model = M.Model([input_embd_layer, input_layer, input_site_layer], 
                    [output_layer_1, output_layer_2])

    model.compile(optimizer=tf.optimizers.Adam(lr=0.002),
                  loss='mse', metrics=['mse'])

    return model

In [22]:
score_df = pd.DataFrame()
predictions = list()

oof_xy = list()
oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
preds_x, preds_y = 0, 0
preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

for fold, (trn_idx, val_idx) in enumerate(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, 
                                                          random_state=SEED).split(data.loc[:, 'path'], 
                                                                                   data.loc[:, 'path'])):
    
    X_train = data.loc[trn_idx, BSSID_FEATS + RSSI_FEATS + ['site_id']] #1 +['floor']
    y_trainx = data.loc[trn_idx, 'x']
    y_trainy = data.loc[trn_idx, 'y']
    y_trainf = data.loc[trn_idx, 'floor']
    
    #X_train = pd.concat([data.loc[trn_idx, BSSID_FEATS + RSSI_FEATS + ['site_id']],
    #                    train_only_data[BSSID_FEATS + RSSI_FEATS + ['site_id']]], axis=0).reset_index(drop=True)
    #y_trainx = pd.concat([data.loc[trn_idx, 'x'], train_only_data['x']], axis=0).reset_index(drop=True)
    #y_trainy = pd.concat([data.loc[trn_idx, 'y'], train_only_data['y']], axis=0).reset_index(drop=True)
    #y_trainf = pd.concat([data.loc[trn_idx, 'floor'], train_only_data['floor']], axis=0).reset_index(drop=True)

    tmp = pd.concat([y_trainx, y_trainy], axis=1)
    y_train = [tmp, y_trainf]

    X_valid = data.loc[val_idx, BSSID_FEATS + RSSI_FEATS + ['site_id']] #2 +['floor']
    y_validx = data.loc[val_idx, 'x']
    y_validy = data.loc[val_idx, 'y']
    y_validf = data.loc[val_idx, 'floor']

    tmp = pd.concat([y_validx, y_validy], axis=1)
    y_valid = [tmp, y_validf]

    model = create_model([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS], X_train.loc[:,'site_id']]) #3 +['floor']
    model.fit([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS], X_train.loc[:,'site_id']], y_train,  #4 +['floor']
                validation_data=([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS], X_valid.loc[:,'site_id']], y_valid), 
                batch_size=batch_size, epochs=epochs, #5+['floor']
                callbacks=[
                ReduceLROnPlateau(monitor='val_xy_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min')
                , ModelCheckpoint(f'{base_path}/RNN_{SEED}_{fold}.hdf5', monitor='val_xy_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')
                , EarlyStopping(monitor='val_xy_loss', min_delta=1e-4, patience=5, mode='min', baseline=None, restore_best_weights=True)
            ])

    model.load_weights(f'{base_path}/RNN_{SEED}_{fold}.hdf5')     #6+['floor']
    val_pred = model.predict([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS], X_valid.loc[:,'site_id']])

    oof_x[val_idx] = val_pred[0][:,0]
    oof_y[val_idx] = val_pred[0][:,1]
    oof_f[val_idx] = val_pred[1][:,0].astype(int)
                                                                  #7+['floor']
    pred = model.predict([test_data.loc[:,BSSID_FEATS], test_data.loc[:,RSSI_FEATS], test_data.loc[:,'site_id']])
    preds_x += pred[0][:,0]
    preds_y += pred[0][:,1]
    preds_f_arr[:, fold] = pred[1][:,0].astype(int)

    score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                        y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
    print(f"fold {fold}: metrics {score}")
    score2 = xy_comp_metric(oof_x[val_idx], oof_y[val_idx],
                        y_validx.to_numpy(), y_validy.to_numpy())
    print(f"fold {fold}: root mean xy position error {score2}")
    oof_xy.append(score2)

    K.clear_session()

preds_x /= (fold + 1)
preds_y /= (fold + 1)
    
print("*+"*40)
score = comp_metric(oof_x, oof_y, oof_f, data.iloc[:, -5].to_numpy(), data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy())
score2 = xy_comp_metric(oof_x, oof_y, data.iloc[:, -5].to_numpy(), data.iloc[:, -4].to_numpy())
print(f"metrics {score}")
print(f"root mean xy position error {score2}")
print(oof_xy)
print("*+"*40)

preds_f_mode = stats.mode(preds_f_arr, axis=1)
preds_f = preds_f_mode[0].astype(int).reshape(-1)
test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
test_preds.columns = subm.columns
test_preds.index = test_data["site_path_timestamp"]
test_preds["floor"] = test_preds["floor"].astype(int)
predictions.append(test_preds)

Epoch 1/40
1115/1115 [==============================] - 81s 69ms/step - loss: 2325.7880 - xy_loss: 2322.5884 - floor_loss: 3.2008 - xy_mse: 2322.5884 - floor_mse: 3.2008 - val_loss: 105.1844 - val_xy_loss: 101.9772 - val_floor_loss: 3.2073 - val_xy_mse: 101.9772 - val_floor_mse: 3.2073

Epoch 00001: val_xy_loss improved from inf to 101.97721, saving model to /kaggle/RNN_2021_0.hdf5
Epoch 2/40
1115/1115 [==============================] - 79s 71ms/step - loss: 164.9417 - xy_loss: 161.7357 - floor_loss: 3.2061 - xy_mse: 161.7357 - floor_mse: 3.2061 - val_loss: 66.8503 - val_xy_loss: 63.6430 - val_floor_loss: 3.2073 - val_xy_mse: 63.6430 - val_floor_mse: 3.2073

Epoch 00002: val_xy_loss improved from 101.97721 to 63.64297, saving model to /kaggle/RNN_2021_0.hdf5
Epoch 3/40
1115/1115 [==============================] - 77s 69ms/step - loss: 126.6985 - xy_loss: 123.4859 - floor_loss: 3.2125 - xy_mse: 123.4859 - floor_mse: 3.2125 - val_loss: 51.1747 - val_xy_loss: 47.9674 - val_floor_loss: 3.2

In [23]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)

lgb_pred = pd.read_csv('../input/indoorlgbfloor/submission.csv')
all_preds['floor'] = lgb_pred['floor'].values

# postprocess (cost minimization)

In [24]:
INPUT_PATH = '../input/indoor-location-navigation'

def compute_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    return rel_positions

def correct_path(args):
    path, path_df = args
    
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['x', 'y']].values
    
    example = read_data_file(f'{INPUT_PATH}/test/{path}.txt')
    rel_positions = compute_rel_positions(example.acce, example.ahrs)
    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)

    N = xy_hat.shape[0]
    delta_t = np.diff(T_ref)
    alpha = (8.1)**(-2) * np.ones(N)
    beta  = (0.3 + 0.3 * 1e-3 * delta_t)**(-2)
    A = scipy.sparse.spdiags(alpha, [0], N, N)
    B = scipy.sparse.spdiags( beta, [0], N-1, N-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

    Q = A + (D.T @ B @ D)
    c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
    xy_star = scipy.sparse.linalg.spsolve(Q, c)
    
    return pd.DataFrame({
        'site_path_timestamp' : path_df['site_path_timestamp'],
        'floor' : path_df['floor'],
        'x' : xy_star[:, 0],
        'y' : xy_star[:, 1],
    })

In [25]:
sub = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
sub["floor"] = all_preds['floor'].values
sub["x"] = all_preds['x'].values
sub["y"] = all_preds['y'].values

tmp = sub['site_path_timestamp'].apply(lambda s : pd.Series(s.split('_')))
sub['site'] = tmp[0]
sub['path'] = tmp[1]
sub['timestamp'] = tmp[2].astype(float)

processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(correct_path, sub.groupby('path'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
sub = pd.concat(dfs).sort_values('site_path_timestamp')
sub.to_csv('submission.csv', index=False)

626it [12:51,  1.23s/it]


# postprocess (snap to grid)

In [26]:
# def split_col(df):
#     df = pd.concat([
#         df['site_path_timestamp'].str.split('_', expand=True) \
#         .rename(columns={0:'site',
#                          1:'path',
#                          2:'timestamp'}),
#         df
#     ], axis=1).copy()
#     return df

# def add_xy(df):
#     df['xy'] = [(x, y) for x,y in zip(df['x'], df['y'])]
#     return df

# def closest_point(point, points):
#     """ Find closest point from a list of points. """
#     return points[cdist([point], points).argmin()]

# train_waypoints = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')

# sub = split_col(sub)
# sub = add_xy(sub)
# train_waypoints = add_xy(train_waypoints)

# ds = []
# for (site, myfloor), d in sub.groupby(['site','floor']):
#     true_floor_locs = train_waypoints.loc[(train_waypoints['floor'] == myfloor) &
#                                           (train_waypoints['site'] == site)] \
#         .reset_index(drop=True)
#     if len(true_floor_locs) == 0:
#         print(f'Skipping {site} {myfloor}')
#         continue
#     d['matched_point'] = [closest_point(x, list(true_floor_locs['xy'])) for x in d['xy']]
#     d['x_'] = d['matched_point'].apply(lambda x: x[0])
#     d['y_'] = d['matched_point'].apply(lambda x: x[1])
#     ds.append(d)

# sub = pd.concat(ds)

In [27]:
# def snap_to_grid(sub, threshold):
#     """
#     Snap to grid if within a threshold.
    
#     x, y are the predicted points.
#     x_, y_ are the closest grid points.
#     _x_, _y_ are the new predictions after post processing.
#     """
#     sub['_x_'] = sub['x']
#     sub['_y_'] = sub['y']
#     sub.loc[sub['dist'] < threshold, '_x_'] = sub.loc[sub['dist'] < threshold]['x_']
#     sub.loc[sub['dist'] < threshold, '_y_'] = sub.loc[sub['dist'] < threshold]['y_']
#     return sub.copy()

# # Calculate the distances
# sub['dist'] = np.sqrt( (sub.x-sub.x_)**2 + (sub.y-sub.y_)**2 )

# sub_pp = snap_to_grid(sub, threshold=4)

# sub_pp = sub_pp[['site_path_timestamp','floor','_x_','_y_','site','path']] \
#     .rename(columns={'_x_':'x', '_y_':'y'})
# sub_pp[['site_path_timestamp','floor','x','y']].to_csv('submission.csv', index=False)